<a href="https://colab.research.google.com/github/roudhasalem8/UAE-Construction-Helmet-Violation-Detection-YOLOv8/blob/main/UAE_Construction_Helmet_Violation_Detection_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install & Import Everything
!pip install ultralytics supervision albumentations -q

import os, shutil, zipfile, cv2, torch
import supervision as sv
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab import drive
print("Ready!")

Ready!


In [ ]:
# Force GPU usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

#Mount Drive & Extract Dataset
drive.mount('/content/drive')


zip_path = "/content/drive/MyDrive/Hard Hat Workers.v3-v3.yolov8.zip"

shutil.copy(zip_path, "/content/dataset.zip")

if os.path.exists("dataset"): shutil.rmtree("dataset")
with zipfile.ZipFile("/content/dataset.zip", 'r') as f:
    f.extractall("dataset")

print("Dataset extracted!")

Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted!


In [ ]:
#Create valid folder
os.makedirs("dataset/valid/images", exist_ok=True)
os.makedirs("dataset/valid/labels", exist_ok=True)

!cp dataset/test/images/* dataset/valid/images/ 2>/dev/null
!cp dataset/test/labels/* dataset/valid/labels/ 2>/dev/null

yaml_content = """
path: /content/dataset
train: train/images
val: valid/images
test: test/images
nc: 3
names:
  0: head     # ← No helmet (DANGER)
  1: helmet   # ← Has helmet (SAFE)
  2: person
"""

with open("/content/dataset/data.yaml", "w") as f:
    f.write(yaml_content.strip() + "\n")

print("data.yaml fixed correctly!")

data.yaml fixed correctly!


In [ ]:
# CELL 4: Check GPU
print("GPU:", torch.cuda.is_available())
print("Name:", torch.cuda.get_device_name(0))

GPU: True
Name: Tesla T4


In [ ]:
# CELL 5: TRAIN THE MODEL
model = YOLO("yolov8m.pt")

model.train(
    data="/content/dataset/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    device=0,
    name="UAE_Helmet_Detection",
    project="/content/runs",
    patience=0,         # no early stopping
    optimizer="AdamW",
    lr0=0.001,
    augment=True,
    conf=0.25,
    iou=0.6,
    plots=True,
    save=True,
    exist_ok=True,
    resume=False        # start fresh for speed
)

print("Training Done!")

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=8, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.6, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=UAE_Helmet_Detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=0, perspective=0.0, plots=True,

In [ ]:
# Load Best Model
best_model = YOLO("/content/runs/UAE_Helmet_Detection/weights/best.pt")
print("Best model loaded!")

In [ ]:
# Test on Your Image
!wget -q "https://i.imgur.com/8k5iH9p.jpg" -O test.jpg

# Run inference
results = best_model("test.jpg", conf=0.4)[0]
detections = sv.Detections.from_ultralytics(results)

# Show normal detection
sv.plot_image(results.plot(), (14, 10))

In [ ]:
# SMART "NO HELMET" ALERT
img = results.orig_img.copy()

# Separate heads and helmets
head_boxes = [box for i, box in enumerate(detections.xyxy) if detections.class_id[i] == 0]
helmet_boxes = [box for i, box in enumerate(detections.xyxy) if detections.class_id[i] == 1]

no_helmet = []

for h_box in head_boxes:
    hx1, hy1, hx2, hy2 = map(int, h_box)
    head_cx = (hx1 + hx2) // 2
    head_cy = (hy1 + hy2) // 2

    has_helmet = False
    for helm_box in helmet_boxes:
        hx1h, hy1h, hx2h, hy2h = map(int, helm_box)
        if hx1h < head_cx < hx2h and hy1h < head_cy < hy2h + 60:
            has_helmet = True
            break

    if not has_helmet:
        no_helmet.append(h_box)

# Draw alerts
alert_img = img.copy()
for box in no_helmet:
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(alert_img, (x1, y1), (x2, y2), (0, 0, 255), 5)
    cv2.putText(alert_img, "NO HELMET!", (x1, y1-20),
                cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 4)

print(f"ALERT: {len(no_helmet)} worker(s) WITHOUT helmet")
sv.plot_image(alert_img, (16, 10))

In [ ]:
# Export + Download Everything
best_model.export(format="onnx", imgsz=640, simplify=True)

!zip -r UAE_Helmet_Final_Result.zip \
  /content/runs/UAE_Helmet_Detection/weights/best.pt \
  /content/runs/UAE_Helmet_Detection/weights/best.onnx \
  test.jpg

from google.colab import files
files.download("UAE_Helmet_Final_Result.zip")